In [ ]:
import pandas as pd
from collections import OrderedDict
import requests

In [ ]:
import boto3
comprehend = boto3.client('comprehend', region_name='us-east-1')

In [ ]:
df = pd.read_excel('Copy of SFPCC Anecdotes and Recordings for Anastasia.xlsx')
df

In [ ]:
all_transcripts = list(df['Transcript'])
#all_transcripts

In [ ]:
f = open("comprehend_experiment.txt", "a")

for transcript in all_transcripts:
    f.write('---------------------TRANSCRIPT------------------\n')
    f.write(transcript)
    # Key phrases
    phrases = comprehend.detect_key_phrases(Text=transcript, LanguageCode='en')

    # Entities
    entities = comprehend.detect_entities(Text=transcript, LanguageCode='en')

    #Sentiments
    sentiments = comprehend.detect_sentiment(Text=transcript, LanguageCode='en')


    # Write the phrases:
    f.write('--------------------- PHRASES ----------------------\n')
    for i in range(0, len(phrases['KeyPhrases'])):
        f.write((phrases['KeyPhrases'][i]['Text']) + "\n")


    # Write the entities with entitity type:
    f.write('------------ ENTITY : ENTITY TYPE -------------\n')
    for i in range(0, len(entities['Entities'])):
        f.write(entities['Entities'][i]['Text'] + ' : ' + entities['Entities'][i]['Type'] + '\n')

    # Write the sentiment:
    f.write('------------------ SENTIMENT -------------------\n')
    f.write(sentiments['Sentiment'] + '\n')
    f.write('------------------------------------------------\n\n\n\n\n')

f.close()

In [ ]:
# natural language processing: n-gram ranking
import re
import unicodedata
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
import matplotlib.pyplot as plt

In [ ]:
ADDITIONAL_STOPWORDS = ['like', 'oh']
def basic_clean(text):
    """
    A simple function to clean up the data. All the words that
    are not designated as a stop word is then lemmatized after
    encoding and basic regex parsing are performed.
    """
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english') + ADDITIONAL_STOPWORDS
    text = (unicodedata.normalize('NFKD', text)
    .encode('ascii', 'ignore')
    .decode('utf-8', 'ignore')
    .lower())
    words = re.sub(r'[^\w\s]', '', text).split()
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

In [ ]:
words = basic_clean(''.join(str(df['Transcript'].tolist())))

In [ ]:
(pd.Series(nltk.ngrams(words, 2)).value_counts())[:10]

In [ ]:
bigrams_series = (pd.Series(nltk.ngrams(words, 2)).value_counts())[:50]
trigrams_series = (pd.Series(nltk.ngrams(words, 3)).value_counts())[:50]

In [ ]:
bigrams_series.sort_values().plot.barh(color='blue', width=.9, figsize=(12, 8))
#plt.show()
plt.savefig('bigrams.png')

In [ ]:
trigrams_series.sort_values().plot.barh(color='blue', width=.9, figsize=(12, 8))
#plt.show()
plt.savefig('trigrams.png')

In [ ]:
quadgrams_series = (pd.Series(nltk.ngrams(words, 4)).value_counts())[:50]
quadgrams_series.sort_values().plot.barh(color='blue', width=.9, figsize=(12, 8))
#plt.show()
plt.savefig('quadgrams.png')

In [ ]:
!pip install wordcloud
from wordcloud import WordCloud
words_list = ' '.join(words)
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                #stopwords = stopwords,
                min_font_size = 10).generate(words_list)

# plot the WordCloud image
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.savefig('wordcloud.png')

In [ ]:
examples_df = pd.read_excel('Copy of Breakdown Spreadsheet for Anastasia.xlsx')
examples_df = examples_df.iloc[:34]
examples_df.columns = examples_df.iloc[0]
examples_df

In [ ]:
example_words = basic_clean(''.join(str(examples_df['Vignette'].tolist())))

In [ ]:
(pd.Series(nltk.ngrams(example_words, 2)).value_counts())[:10]
bigrams_series = (pd.Series(nltk.ngrams(example_words, 2)).value_counts())[:50]
trigrams_series = (pd.Series(nltk.ngrams(example_words, 3)).value_counts())[:50]

In [ ]:
bigrams_series.sort_values().plot.barh(color='blue', width=.9, figsize=(12, 8))
plt.show()

In [ ]:
trigrams_series.sort_values().plot.barh(color='blue', width=.9, figsize=(12, 8))
plt.show()

In [ ]:
from wordcloud import WordCloud
example_words_list = ' '.join(example_words)
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                #stopwords = stopwords,
                min_font_size = 10).generate(example_words_list)

# plot the WordCloud image
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.savefig("examples wordcloud.png")